# Using Spark to Parse Text Files With a Custom Delimiter

Suppose you have a text file with data that uses non-standard delimiters.  Can we still use Spark to parse that data?  Yes!  We just have to use some custom string maniuplation code.
That won't be quite as efficient as using native Spark, but it will get the job done!

Let's make a file that has data where some fields use a pipe delimiter `|` and other fields use a combination of pipe and caret `|^my data value^|`

In [0]:
# Make a small data file

testData = """Name|Color|Size|Origin
Rabbit^|^Brown^|7|^Warehouse
Horse^|^Green^|28|^Dock
Pig^|^Orange^|17|^Port
Cow^|^Blue^|23|^Warehouse
Bird^|^Yellow^|2|^Dock
Dog^|^Brown^|10|^Port
Carrot^Man^|^Orange^|22|^Warehouse
"""

dbutils.fs.put("data.txt", testData, True)

Wrote 205 bytes.
Out[1]: True

First, let's just read in the data using the `|` as a separator.  This should work, but some of our fields will start and end with a caret.

In [0]:
display( spark.read.option("header", "true").option("sep", "|").csv("/data.txt") )

Name,Color,Size,Origin
Rabbit^,^Brown^,7,^Warehouse
Horse^,^Green^,28,^Dock
Pig^,^Orange^,17,^Port
Cow^,^Blue^,23,^Warehouse
Bird^,^Yellow^,2,^Dock
Dog^,^Brown^,10,^Port
Carrot^Man^,^Orange^,22,^Warehouse


### Option 1:  Split the Data and Remove the Carets

One option is to split the data (as we did in our quick example above) and then loop through each column, removing any carets at the beginning and end of the data.

In [0]:
# Option 1
dff = spark.read.option("header", "true").option("inferSchema", "true").option("delimiter", "|").csv("/data.txt")

from pyspark.sql.functions import regexp_replace

dffs_headers = dff.dtypes
for i in dffs_headers:
    columnLabel = i[0]
    newColumnLabel = columnLabel.replace('^','').replace('^','') 
    dff = dff.withColumn(newColumnLabel, regexp_replace(columnLabel, '^\\^|\\^$', ''))
    if columnLabel != newColumnLabel:
      dff = dff.drop(columnLabel)
    
display(dff)

Name,Color,Size,Origin
Rabbit,Brown,7,Warehouse
Horse,Green,28,Dock
Pig,Orange,17,Port
Cow,Blue,23,Warehouse
Bird,Yellow,2,Dock
Dog,Brown,10,Port
Carrot^Man,Orange,22,Warehouse


### Option 2: Replace Carets Near Pipes Then Split

A different approach is to read each line of the text file as a row in an RDD.  Then replace any occurances of `|^` or `^|` with a plain pipe.  Then we can split the data based on the pipe.

Note that with this approach, we have no way to tell Spark that our first row contains column headers.  So we do an extra step at the end where we filter out the rows that contain
the header information.  We then reapply that header information when we convert the RDD to a Spark DataFrame.

In [0]:
rdd = (sc
         .textFile("data.txt")
         .map(lambda x: x.replace('|^','|').replace('^|','|').split('|'))
      )

header = rdd.first()

dff = rdd.filter(lambda row : row != header).toDF(header)

display(dff)

Name,Color,Size,Origin
Rabbit,Brown,7,Warehouse
Horse,Green,28,Dock
Pig,Orange,17,Port
Cow,Blue,23,Warehouse
Bird,Yellow,2,Dock
Dog,Brown,10,Port
Carrot^Man,Orange,22,Warehouse
